# Classification binaire avec la régression logistique


Dans ce TP, nous allons apprendre à mettre en oeuvre un modèle de régression logistique pour la prédiction de labels binaires $y$.   
Rappel : la régression logistique repose sur une modélisation probabiliste, et donc plutôt que de prédire simplement 0/1, 
on calcule pour  chaque nouvelle observation un  score  entre 0 et 1 qui correspond à un estimateur de la probabilité $\mathbb P(y=1|x)$. 
 
## Données *score*

Le tableau *score* contient des données sur la solvabilité d'anciens clients en vue d'un crédit pour l'achat d'un véhicule.  La colonne `Y` indique si le client était solvable ($y=0$) ou pas ($y=1$) et c'est le label à prédire. Les autres colonnes du tableau sont des covariables avec des informations supplémentaires sur les clients. 
On cherche à construire un modèle qui explique la solvabilité en fonction de ces covariables : on veut donc apprendre la relation précise entre les covariables et le label à prédire.

Il n'est pas clair si toutes les covariables sont utiles pour cette tâche, donc dans un second temps, on se posera la question de la sélection des covariables qui servent à prédire.

## Préparer les données
### Charger les données à partir du fichier externe
On commence par charger les données stockées dans un fichier *day1_score.csv*. Si le fichier n'est pas trop gros, vous pouvez commencer par ouvrir ce fichier **avec un simple éditeur de texte**, pour identifier sa structure et paramétrer son importation. Vous constaterez que la première ligne indique les noms des variables, le séparateur est `;` et les décimales sont codées par des virgules (et non des points). Sinon, importation avec les paramètres standards et itérations successives pour trouver le bon format ! 

On peut simplement afficher les premières lignes avec la commande suivante afin de savoir comment paramétrer la fonction d'importation des données à partir du fichier :

In [ ]:
%%bash
head -2 'data/day1_score.csv'

In [ ]:
import os
import numpy as np
import pandas as pd

# Votre chemin vers les données. Par défaut celui du notebook 
# path_data = '.'
path_data = 'data/' 

# Lecture du fichier csv
df = pd.read_csv(os.path.join(path_data, 'day1_score.csv'), sep=';', decimal=',')

### Exercice 1

- Familiarisez-vous avec les données (à l'aide de `shape`, `head`, `dtypes`, `describe`, `iloc`,`items`). 
- Séparer les données en 3 dataframes qu'on nomme :

    - `defaut` : pour les labels observés (`Y`)
    - `quanti`: dataframe avec toutes les covariables quantitatives
    - `quali`: dataframe avec toutes les covariables qualitatives

In [ ]:
# Your answer

### Réponse

In [ ]:
df.shape

Le jeu de données contient 10 000 lignes (individus) décrits par 21 variables (colonnes).  

In [ ]:
df.head()

In [ ]:
df.dtypes

La première variable `Y` est la variable binaire à prédire. Il y a ensuite 10 variables quantitatives (`X1` à `X10`) et 10 variables qualitatives (`X11` à `X20`).

Résumé des variables quantitatives :

In [ ]:
df.describe()

Si on souhaite un résumé incluant les variables qualitatives :

In [ ]:
# Quelques stats descriptives, quantitatives et qualitatives, met des NaN si incongru.
df.describe(include='all')

Noter que `freq` indique le comptage de la modalité la plus fréquente.

In [ ]:
# Combien de modalités pour chaque variable quali ?
for colname, col in df.iloc[:, 11:21].items():  # .items() permet d'itérer sur les noms des colonnes et leurs contenus
    print(colname + ':')
    print(col.value_counts())
    print('-' * 16)

Séparons les différents types de variables :

In [ ]:
# Identification selon le type de variable
defaut = df['Y'] 
quanti = df.iloc[:, 1:11]
quali = df.iloc[:, 11:21]

### Standardisation des données quantitatives et codage des qualitatives

On ne travaille que très rarement sur des donnnées brutes. 

Pour des  **variables quantitatives**, il est important des les  **standardiser** (centrer et réduire) afin de les ramener à  des échelles comparables. Cela diminue également les problèmes numériques. 

Pour une **variable qualitative** ou **catégorielle**, il n'y a en général pas  d'ordre naturel de ses modalités. Donc il ne faut pas bêtement remplacer des catégories *A*, *B*, *C*,... par 1, 2, 3,... Le problème est que dans un modèle de régression, on va calculer par exemple des moyennes sur ces valeurs, mais en général *B* n'est pas la moyenne de *A* et *C* (alors que c'est le cas pour la valeur 2 par rapport à 1 et 3). Il nous faut alors un encodage qui est invariant à l'ordre des modalités. On utilise  le **one-hot encoding** qui consiste à transformer une variable qualitative en plusieurs variables binaires dites **dummies**. Une variable avec K modalités est transformée en K-1 variables binaires. Chacune de ces variables binaires indique pour une catégorie spécifiée si la variable observée est égale à cette catégorie ou pas. Il suffit de K-1 variables binaires pour encoder K catégories, car l'information sur la K-ième catégorie peut être déduite des K-1 autres variables binaires. Donc, si on utilisait K variables binaires, on introduirait des corrélations entre les colonnes, ce qu'il faut éviter dans un cadre de régression.

### Exercice 2

- Standardiser les données quantitatives.
- Créer des dummies pour toutes les variables qualitatives avec la fonction `get_dummies` de `pandas` et l'option `drop_first=True`.
- Créer un seul dataframe qu'on nommera `df_trav` contenant toutes les covariables transformées (quantitatives et qualitatives).

In [ ]:
# Your answer

### Réponse

In [ ]:
# Reduction des variables quantitatives

# Méthode directe 
quantiN = (quanti-quanti.mean())/quanti.std()

## Méthode sophistiquée avec StandardScaler
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#quantiN = pd.DataFrame(sc.fit_transform(quanti), columns=quanti.columns)

# Nouveau dataframe avec les var quanti normalisées
df_trav = pd.concat([quantiN, quali], axis=1) 


In [ ]:
# On transforme les variables qualitatives en dummies
df_trav = pd.get_dummies(df_trav, drop_first=True)  # .getdummies() n'agit que sur les var qualitatives et garde les quantitatives
# Ne pas oublier 'drop_first=True' sinon on garde K dummies pour chaque variable qui a K modalités ! 
df_trav.head()

In [ ]:
# Noter qu'à présent toutes les variables sont quantitatives (plus besoin de l'option include="all" dans la fonction `describe`)
df_trav.describe()

In [ ]:
df_trav.dtypes

In [ ]:
# On enregistre les noms des colonnes
features_names = df_trav.columns.tolist()
print(features_names)

On peut éventuellement sauvegarder notre jeu de données. Il est prêt à être analysé.

In [ ]:
## Si on veut sauvegarder ce jeu de données dans un fichier

#import pickle as pkl

## Sauver le jeu de données dans un fichier 
#with open(os.path.join(path_travail, 'df_dumN.pkl'), 'wb') as f:
#    pkl.dump({'features': df_trav, 'labels': defaut}, f)

### Échantillons d'apprentissage et de test

In [ ]:
# On appelle les données : features X et variables à prédire Y 
X, Y = df_trav, defaut

# Si on veut recharger le jeu de données depuis le fichier sauvegardé
# Chargement base de travail
#with open(os.path.join(path_travail, 'df_dumN.pkl'), 'rb') as f:
#    data = pkl.load(f)  
#X, Y = data['features'], data['labels']


On rappelle que la modélisation se fait en trois temps :
- on sépare les données : TRAIN / TEST
- on apprend le modèle sur TRAIN
- on évalue la performance du modèle appris sur TEST

On coupe les données de façon aléatoire en deux groupes. 
Le plus souvent, on les sépare  en 80% pour l'apprentissage et 20% pour le test. D'autres pourcentages courants sont 67-33 ou 50-50.
On   utilise la  fonction `train_test_split` du package `sklearn.model_selection` pour séparer aléatoirement les données. 

Même si le split est aléatoire, on souhaite que les deux échantillons soient tous les deux représentatifs du problème. En particulier, on voudrait qu'ils contiennent le même pourcentage de labels 0 et 1, ce qui est notamment important quand  les labels ne sont pas équilibrés (pas 50-50).

Ici, le taux de défaut, i.e. la proportion de clients qui n'ont pas remboursé leur crédit (y=1) dans la variable à prédire `Y` dans l'échantillon global est :

In [ ]:
# Calculons le taux de cible
tx_cible = Y.mean()
# Affichons le taux de cible joliment
print("Taux de cible: {taux:.2f}%".format(taux=100 * tx_cible))

Le taux de défaut est bas, donc une coupe totalement aléatoire des données risque de produire des sous-échantillons où l'un des deux ne contient que  peu de labels qui valent 1. Cela peut être problématique pour l'apprentissage du modèle comme pour l'évaluation de la méthode.
En pratique, dans des problèmes de classification, on force alors la même répartition des labels dans les deux échantillons TRAIN et TEST, via l'option `stratify`.

### Exercice 3

Utiliser la  fonction `train_test_split` du package `sklearn.model_selection` pour séparer les données en `train` et `test` avec 80% pour les données `train` et 20% pour les données `test`. On force la même répartition des labels dans les deux sous-échantillons.
**Pour ce TP** fixons la graine du générateur aléatoire (via l'option `random_state=19`) afin que nous travaillons tous sur les mêmes données `train` et `test`. En effet,  la sélection de variables par pénalisation $\ell_1$ est très instable et dépend énormément du split des données effectué. 

In [ ]:
# Your answer

### Réponse

In [ ]:
from sklearn.model_selection import train_test_split

# split TRAIN / TEST
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=77)

In [ ]:
print(Y_train.mean(), Y_test.mean())

On voit que les échantillons  `train` et `test` ont exactement le même taux de défaut.

## Apprentissage par régression logistique 

Pour commencer, on va apprendre, sur les données `train`, les coefficients de la régression logistique de `Y` sur l'ensemble des variables `X` (soit 32 variables). 

### Exercice 4
- Utiliser la fonction `LogisticRegression` de `sklearn.linear_model` pour effectuer une régression logistique simple (i.e. sans pénalisation). Visualiser les coefficients estimés (intercept et variables).
- Puis sur le jeu `test`, obtenez les probabilités de défaut de chaque individu (`predict_proba()`) et les prédictions (`predict()`) pour le seuil par défaut $t=1/2$. Utiliser la fonction `score` pour calculer le pourcentage de bien classés (`accuracy`). 

In [ ]:
# Your answer

### Réponse

In [ ]:
from sklearn.linear_model import LogisticRegression

# Définition du modèle de régression logistique : sans pénalisation 
logreg = LogisticRegression(penalty='none', max_iter=1000) # sans l'option max_iter, on peut avoir un nb max d'itérations insuffisant pour atteindre la convergence
# On estime les paramètres de ce modèle sur les données 
result = logreg.fit(X_train, Y_train)

# Résultats de la regression logistique
# On visualise l'intercept b et les 32 coefficients w_i estimés pour les 32 variables 
print(result.intercept_)
print(result.coef_)

In [ ]:
# Probabilités de non-défaut (y=0) et de défaut (y=1) pour chaque individu dans le jeu test 
result.predict_proba(X_test)

On obtient deux colonnes qui donnent les probas prédites pour les deux labels 0 et 1.
On peut constater que pour certains individus, la prédiction est très tranchée (93% vs 7%) tandis que pour d'autres, la prédiction est beaucoup plus incertaine (52% vs 48%).

In [ ]:
# Prédiction du label sur nos données test - ici on applique le seuil t=1/2 sur les probas ci-dessus
Y_pred =result.predict(X_test) 
print(Y_pred)
print(Y_pred.mean())

On constate qu'on prédit seulement 4,5% de défaut, alors que les jeux de données ont été stratifiés, donc on sait qu'il y a 15% de défaut dans les données TEST.

In [ ]:
# Accuracy (i.e. pourcentage de bien classés) sur les données de test
result.score(X_test,Y_test)

## Autre façon d'obtenir la même chose :
#from sklearn.metrics import accuracy_score

#accuracy_score(Y_test,Y_pred)

L'accuracy est bonne alors qu'on a peu de défaut prédit : c'est parce que les 0 sont très nombreux ! Un prédicteur constant à 0 donne une accuracy de 85% !

In [ ]:
# Matrice de confusion 
from sklearn import metrics

metrics.confusion_matrix(Y_test, Y_pred)

### Courbes ROC et precision-recall
À présent on veut regarder les performances du classifieur en faisant jouer des rôles non symétriques aux valeurs 0 et 1. En effet pour une banque, il est plus important d'identifier les vrais négatifs ($y=0$ et $\hat y=0$ i.e. les clients à qui on va octroyer un crédit et qui vont effectivement le rembourser) alors qu'on peut se permettre d'avoir des faux positifs ($y=0$ et $\hat y =1$ i.e. des clients à qui on n'accorde pas le crédit alors qu'ils l'auraient remboursé). 

Noter que du point de vue du client, les priorités ne sont pas les mêmes ! De même si on s'intéressait à un organisme social de micro-crédit, on voudrait au contraire limiter les faux positifs. Suivant le problème considéré toutes ces quantités (TP, FP, TN, FN) n'ont pas la même importance. 

### Exercice 5

- Tracer les courbes ROC et precision-recall obtenues sur le jeu `test`. 
- À titre de comparaison, on tracera aussi les mêmes courbes obtenues sur le jeu de données `train`. On rappelle que les performances du classifieur doivent toujours être calculées à partir d'un échantillon `test` indépendant du jeu de données `train` qui a servi à la construction du classifieur. Cependant il est utile de comparer les courbes ROC calculées avec la base `test` versus la base `train` pour détecter un sur-apprentissage potentiel. 
- On utilisera la librairie `matplotlib.pyplot` pour tracer les graphiques et les fonctions `roc_curve` et `auc` de `sklearn.metrics` pour calculer les points de la courbe ROC et l'AUC.  


In [ ]:
# Your answer

### Réponse

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# COURBES ROC ET AUC 

fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

# Sur le jeu de TEST, on reprend les scores prédits (probas que Y=1) 
scores_test= logreg.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(Y_test, scores_test) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, label=f"Test - AUC = %0.2f" % auc(fpr, tpr))
print('AUC sur test=%.2f' %metrics.auc(fpr, tpr))

# La même courbe mais sur le jeu TRAIN
scores_train= logreg.predict_proba(X_train)[:,1]
fpr, tpr, _ = roc_curve(Y_train,scores_train)
ax.plot(fpr, tpr, label=f"Train - AUC = %0.2f" % auc(fpr, tpr))
print('AUC sur train=%.2f' %metrics.auc(fpr, tpr))

fig.legend(loc="center right")
plt.show()

In [ ]:
# Variante pour AUC-ROC
from sklearn.metrics import roc_auc_score 

AUC_test = roc_auc_score(Y_test, scores_test)
print('ROC_AUC_Score sur test =%.2f' %AUC_test)
AUC_train = roc_auc_score(Y_train, scores_train)
print('ROC_AUC_Score sur train=%.2f' %AUC_train)


In [ ]:
# COURBES PRECISION-RECALL

fig, ax = plt.subplots()
ax.plot([0, 1], [1,0], 'k--')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

# Sur le jeu de TEST :
precision, recall, _ = precision_recall_curve(Y_test, scores_test)
# Attention le recall est en abscisses et la précision en ordonnée
ax.plot(recall, precision, label=f"Test - AUC = %0.2f" % metrics.auc(recall,precision)) 
# On calcule l'aire sous la courbe precision-recall exactement comme pour la courbe ROC

# La même courbe mais sur le jeu TRAIN
precision, recall, _ = precision_recall_curve(Y_train,scores_train)
ax.plot(recall, precision, label=f"Train - AUC = %0.2f" % metrics.auc(recall,precision))

fig.legend(loc="center right")
plt.show()

**Remarques** : 
* les courbes ROC et Precision-Recall ne se lisent pas de la même façon.
* Ici on a une AUC-PR très mauvaise. En effet, le classifieur a tendance à prédire beaucoup de 0 et les classes sont non équilibrées. 


## Régression logistique avec pénalité Ridge

Dans cette partie, on va utiliser une pénalité en norme 2 (dite $\ell_2$ ou Ridge) sur les coefficients de la régression. Cette pénalité Ridge a tendance à "rétrecir" les valeurs des coefficients (on parle de shrinkage) et régularise la solution. 

### Exercice 6

- Mettre en oeuvre la régression logistique avec pénalité $\ell_2$ sur les données d'apprentissage. On utilisera toujours la fonction `LogisticRegression`avec  l'option `penalty='l2'`. En plus, on utilisera une pénalité assez forte, p. ex. `C=0.01`. 
- Comparer les valeurs des coefficients avec le cas sans pénalité (visualisez l'effet de shrinkage). 
- Évaluer les performances du classifieur sur les données de test.
- Pour les deux classifieurs (logreg et sa version pénalisée en norme 2), comparer les courbes de score sur les données `test` ainsi que les performances en termes de courbes (ROC ou precision-recall) et d'AUC. 

In [ ]:
# Your answer

### Réponse

In [ ]:
# Définition du modèle de régression logistique : pénalisation norme 2 
logreg_l2 = LogisticRegression(penalty='l2', C=0.01, max_iter=1000)
# On estime les paramètres de ce modèle sur les données 
result_l2 = logreg_l2.fit(X_train, Y_train)

# Resultats de la regression logistique
# On visualise l'intercept b et les 32 coefficients w_i estimés pour les 32 variables - On les compare aux précédents
print(result_l2.intercept_)
print(result.intercept_ )
print('------')
print(result_l2.coef_)
print(result.coef_)

In [ ]:
# Visualisation des différences en valeur absolue

# Declaring the figure (width, height)
plt.figure(figsize=[15, 10])

# Data to be plotted
height = np.abs(np.concatenate(result.coef_)) # np.concatenate transforme un array en vecteur
height_l2 = np.abs(np.concatenate(result_l2.coef_))
variables = X.columns
x_pos = np.arange(len(variables))

# Create bars
plt.bar(x_pos, height, color = 'b', width = 0.25)
plt.bar(x_pos+ 0.25, height_l2, color = 'g', width = 0.25)

# Create names on the x-axis
plt.xticks(x_pos,variables)

# Creating the legend of the bars in the plot
plt.legend(['No penalty', 'norm-2 penalty'])

# Show graphic
plt.show()

On constate l'effet de *shrinkage*. 

In [ ]:
# On trace les courbes de score sur les données de test pour nos deux classifieurs
pred_test_l2 = pd.DataFrame({
    'Y': Y_test, 
    'score' : result_l2.predict_proba(X_test)[:,1], 
    })

pred_test = pd.DataFrame({
    'Y': Y_test, 
    'score' : result.predict_proba(X_test)[:,1], 
    })

# on ordonne les prédictions par score décroissant
pred_test_l2 = pred_test_l2.sort_values(by='score', ascending=False) 
pred_test = pred_test.sort_values(by='score', ascending=False) 

plt.xlabel('individuals (score decreasing order)')
plt.ylabel('score')

# Courbes de scores 
plt.subplot()
x = np.linspace(0, 1, len(pred_test.score))

plt.plot(x, pred_test_l2.score, linewidth=2, color='r', label=f"LogReg-Ridge") 
plt.plot(x, pred_test.score, linewidth=2, color='b', label=f"LogReg (no penalty)") 

# repères 
plt.vlines(0.15,0,1,color='gray',linestyle='dashed') # 15% d'individus avec y=1
plt.hlines(0.5,0,1,color='gray',linestyle='dashed') # seuil t=1/2

plt.legend(loc="upper right")
plt.show()

Ici on constate que les deux prédicteurs sont différents. La pénalisation donne des probabilités moins tranchées pour les deux classes, c'est-à-dire les scores élevés (resp. faibles) sont plus faibles (resp. plus forts) que pour la régression non pénalisée. Les 15% d'individus avec $y=1$ devraient être à gauche de la ligne grise verticale (score de défaut élevé) et la régression pénalisée $\ell_2$ leur attribue un score inférieur à celui donné par la régression simple. La droite grise horizontale correspond au seuil $t=0.5$: dans une approches naive on prédit 1 si la probabilité calculée par le modèle (le score) dépasse 0.5.
   

In [ ]:
# Comparons les deux classifieurs via leurs courbes ROC et les AUC-ROC

fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

# Sur le jeu de TEST, on reprend les scores prédits (probas que Y=1) 
scores_test_l2 = logreg_l2.predict_proba(X_test)[:,1]
Y_pred_l2= logreg_l2.predict(X_test)
fpr, tpr, _ = roc_curve(Y_test, scores_test_l2) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, label=f"LogReg-Ridge - AUC = %0.2f" % auc(fpr, tpr))

scores_test= logreg.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(Y_test, scores_test) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, label=f"LogReg (no penalty) - AUC = %0.2f" % auc(fpr, tpr))

plt.legend(loc="lower right")
plt.show()

Les performances sont les mêmes en termes d'AUC-ROC. Regardons ce qui se passe pour la courbe precision-recall.

In [ ]:
# On compare maintenant les deux classifieurs via leurs courbes PR et les valeurs AUC-PR

fig, ax = plt.subplots()
ax.plot([0, 1], [1,0], 'k--')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

# Sur le jeu de TEST 
# pour la prediction pénalisée en norme 2
precision, recall, _ = precision_recall_curve(Y_test, scores_test_l2)
ax.plot(recall, precision, label=f"LogReg-Ridge - AUC = %0.2f" % metrics.auc(recall,precision)) 

#  pour la prédiction sans pénalité 
precision, recall, _ = precision_recall_curve(Y_test, scores_test) 
ax.plot(recall, precision, label=f"LogReg (no penalty) - AUC = %0.2f" % metrics.auc(recall,precision))

plt.legend(loc="upper right")
plt.show()

Là encore, on a les mêmes performances en termes d'AUC-PR.

## Regression logistique avec sélection de variables

Dans cette partie, on va mettre en oeuvre une pénalité $\ell_1$ dans la régression logistique. Cela va nous permettre de faire de la sélection de variables : les variables peu informatives pour la prédiction ne seront plus du tout utilisées (coefficient $w_i$ estimé à 0). On espère ainsi améliorer les performances de notre prédicteur. 

### Exercice 7

- Mettre en oeuvre la régression logistique avec pénalité $\ell_1$ sur les données d'apprentissage (avec l'option le `solver='liblinear'`). On utilisera une pénalité assez forte (ex $C=0.01$). 
- Comparer les valeurs des coefficients avec le cas sans pénalité (visualisez l'effet d'annulation des coefficients). 
- Évaluer les performances du classifieur sur les données `test`.
- Pour les deux classifieurs (logreg et sa version pénalisée en norme 1), comparer les courbes de score sur les données `test` ainsi que les performances en termes de courbes (ROC et precision-recall) et d'AUC. 

In [ ]:
# Your answer

### Réponse

In [ ]:
# Définition du modèle de régression logistique : pénalisation norme 1 
logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', C=0.01, max_iter=1000)
# On estime les paramètres de ce modèle sur les données 
result_l1 = logreg_l1.fit(X_train, Y_train)

# Resultats de la regression logistique pénalisée en norme 1
# On visualise l'intercept b et les 32 coefficients w_i estimés pour les 32 variables 
print(result_l1.intercept_)
print('------')
print(result_l1.coef_)

On constate qu'un grand nombre de coefficients de la régression pénalisée $\ell_1$ sont mis à 0.

In [ ]:
# Visualisation des différences en valeur absolue

# Initialiser la figure [width, height]
plt.figure(figsize=[15, 10])

# Data to be plotted
height = np.abs(np.concatenate(result.coef_)) # np.concatenate transforme un array en vecteur
height_l2 = np.abs(np.concatenate(result_l2.coef_))
height_l1 = np.abs(np.concatenate(result_l1.coef_))
variables = X.columns
x_pos = np.arange(len(variables))

# Create bars
plt.bar(x_pos, height, color = 'b', width = 0.25)
plt.bar(x_pos+ 0.25, height_l2, color = 'g', width = 0.25)
plt.bar(x_pos+ 0.5, height_l1, color = 'red', width = 0.25)

# Create names on the x-axis
plt.xticks(x_pos,variables)

# Creating the legend of the bars in the plot
plt.legend(['No penalty', 'norm-2 penalty','norm-1 penalty'])

# Show graphic
plt.show()

In [ ]:
# On trace les courbes de score sur les données de test pour nos deux classifieurs (sans pénalité et pénalité en norme 1)

pred_test_l1 = pd.DataFrame({
    'Y': Y_test, 
    'score' : result_l1.predict_proba(X_test)[:,1], 
    })

# on ordonne les prédictions par score décroissant
pred_test_l1 = pred_test_l1.sort_values(by='score', ascending=False) 

plt.xlabel('individuals (score decreasing order)')
plt.ylabel('score')

# Courbes de scores 
plt.subplot()
x = np.linspace(0, 1, len(pred_test.score))

plt.plot(x, pred_test_l1.score, linewidth=2, color='r', label=f"LogReg-L1-penalty") 
plt.plot(x, pred_test_l2.score, linewidth=2, color='green', label=f"LogReg-L2-penalty") 
plt.plot(x, pred_test.score, linewidth=2, color='b', label=f"LogReg (no penalty)") 

# repères 
plt.vlines(0.15,0,1,color='gray',linestyle='dashed') # 15% d'individus avec y=1
plt.hlines(0.5,0,1,color='gray',linestyle='dashed') # seuil t=1/2

plt.legend(loc="upper right")
plt.show()

Encore une fois, on constate que les deux prédicteurs sont différents. La pénalisation donne des probabilités moins tranchées pour les deux classes, i.e.  les scores élevés (resp. faibles) sont plus faibles (resp. plus forts) que pour la régression non pénalisée. Les 15% d'individus avec $y=1$ devraient être à gauche (score de défaut élevé) et la régression pénalisée $\ell_1$ leur attribue un score inférieur à celui donné par la régression simple ou par la régression $\ell_2$.  
   

In [ ]:
# Comparons les classifieurs via leurs courbes ROC et les AUC-ROC

fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

# Sur le jeu de TEST, on reprend les scores prédits (probas que Y=1) 
scores_test_l1= logreg_l1.predict_proba(X_test)[:,1]
Y_pred_l1= logreg_l1.predict(X_test)
fpr, tpr, _ = roc_curve(Y_test, scores_test_l1) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, color='r',label=f"LogReg-L1-penalty - AUC = %0.2f" % auc(fpr, tpr))

# courbe ROC régression simple
fpr, tpr, _ = roc_curve(Y_test, scores_test) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, color='b',label=f"LogReg (no penalty) - AUC = %0.2f" % auc(fpr, tpr))

# courbe ROC régression pénalisée L2
fpr, tpr, _ = roc_curve(Y_test, scores_test_l2) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, color='green',label=f"LogReg (no penalty) - AUC = %0.2f" % auc(fpr, tpr))


plt.legend(loc="lower right")
plt.show()

Les performances sont légèrement moins bonnes en termes d'AUC-ROC. Regardons ce qui se passe pour la courbe precision-recall.

In [ ]:
# On compare maintenant les deux classifieurs via leurs courbes PR et les valeurs AUC-PR

fig, ax = plt.subplots()
ax.plot([0, 1], [1,0], 'k--')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

# Sur le jeu de TEST 
# pour la prediction pénalisée en norme 1
precision, recall, _ = precision_recall_curve(Y_test, scores_test_l1)
ax.plot(recall, precision, color='r',label=f"LogReg-L1-penalty - AUC = %0.2f" % metrics.auc(recall,precision)) 

#  pour la prédiction sans pénalité 
precision, recall, _ = precision_recall_curve(Y_test, scores_test) 
ax.plot(recall, precision, color='b',label=f"LogReg (no penalty) - AUC = %0.2f" % metrics.auc(recall,precision))

# pour la prediction pénalisée en norme 2
precision, recall, _ = precision_recall_curve(Y_test, scores_test_l2)
ax.plot(recall, precision, color='green',label=f"LogReg-L2-penalty - AUC = %0.2f" % metrics.auc(recall,precision)) 

plt.legend(loc="upper right")
plt.show()

Là encore, on a des performances un peu dégradées pour la régression en pénalité $\ell_1$ en termes d'AUC-PR.

### Chemins de régularisation pour la pénalité $\ell_1$

Dans cette partie, on va visualiser l'effet de la **constante de pénalité** sur l'évolution des coefficients $w_i$, à travers les **chemins de régularisation**. Lorsque la pénalité est très forte (i.e $C$ très petite), aucune  variable n'est sélectionnée (tous les coefficients $w_i$ sont estimés à 0). Dans ce cas, la fonction de régression logistique est constante (on a seulement l'intercept) et le lien entre $y$ et $X$ est très mal appris. Puis au fur et à mesure que $C$ augmente, on inclue de plus en plus de variables dans notre modèle de régression logistique. Lorsque $C$ est très grande, on ne pénalise plus et on retrouve les résultats de la régression logistique simple. 

### Exercice 8

Fixer une grille de valeurs pour la constante $C$, fitter le modèle avec ces constantes et visualiser  les chemins de régularisations.

In [ ]:
# On fabrique un ensemble de valeurs C entre 10^(-4) et 10^2
c_path = np.logspace(-4, 2, 10)
# c_path plus long ==> durée calcul plus longue
print(c_path) 


In [ ]:
# Chemin de régularisation 
logregC = LogisticRegression(penalty='l1', solver='liblinear')
coeffs = []
for c in c_path:
    logregC.C = c
    logregC.fit(X_train, Y_train)
    coeffs.append(logregC.coef_.ravel().copy())

coeffs = np.array(coeffs)
plt.figure(figsize=(10, 8))
plt.semilogx(c_path, coeffs)
ymin, ymax = plt.ylim()
plt.xlabel('C', fontsize=16)
plt.xticks(fontsize=14)
plt.ylabel('Coefficients', fontsize=16)
plt.yticks(fontsize=14)
plt.title('Evolution de chaque coef de la régression logistique '
          'avec la penalisation L1', fontsize=16)

plt.show()

## Validation croisée pour le choix de la constante de pénalité 

### Régression logistique avec sélection de variables (pénalité $\ell_1$ + validation croisée)

À chaque fois qu'on utilise une pénalité, il faut choisir la constante de pénalisation. 
Attention : la fonction `LogisticRegression` ne sait pas choisir la constante de pénalisation, qui par défaut est fixée à 1. Ce choix est parfaitement débile (car il ne correspond à rien) et n'a aucune raison d'être utilisé. 

Dans cette partie, on va mettre en oeuvre le choix de la constante de pénalisation par **validation croisée**. 
Nous le ferons dans le cadre d'une pénalité $\ell_1$, mais on pourrait faire exactement la même chose avec un autre type de pénalité. 


### Exercice 9

Utiliser la fonction `LogisticRegressionCV` pour sélectionner automatiquement par validation croisée  la constante $C$ de pénalité dans un modèle de régression logistique avec pénalité $\ell_1$. 

In [ ]:
# Your answer

### Réponse

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# Modèle de régression logistique avec pénalité l1 et validation croisée pour le choix automatique de la constante de pénalité C
logreg_cv = LogisticRegressionCV(penalty='l1',
                                 tol=1e-3,
                                 Cs=np.logspace(-3, 5, 15), # grille de 15 valeurs pour C entre 10^-3 et 10^5
                                 cv=3, # normalement CV avec 10 folds ; ici 3 pour gagner du temps
                                 solver='liblinear',
                                 # class_weight='balanced',
                                 scoring='roc_auc') # le critère pour comparer les différents modèles par CV - Par défaut c'est l'accuracy
logreg_cv.fit(X_train, Y_train)

# résultats intermédiaires de calculs de ROC-AUC sur chacun des cv-folds
crit = logreg_cv.scores_[1]
print(crit)

In [ ]:
np.logspace(-3, 5, 15)

In [ ]:
# Visualisation du critère sur chaque fold pour les différentes valeurs de C

# Declaring the figure (width, height)
plt.figure(figsize=[15, 10])

# X-axis
variables = np.round(np.logspace(-3, 5, 15),2) # on arrondit les valeurs de C affichées
x_pos = np.arange(len(variables))

# Create bars
plt.bar(x_pos, crit[0], color = 'b', width = 0.25)
plt.bar(x_pos+ 0.25, crit[1], color = 'g', width = 0.25)
plt.bar(x_pos+ 0.55, crit[2], color = 'r', width = 0.25)

# Create names on the x-axis
plt.xticks(x_pos,variables)

# Creating the legend of the bars in the plot
plt.legend(['ROC-AUC-1', 'ROC-AUC-2','ROC-AUC-3'])

# Show graphic
plt.show()


Ci-dessus pour chacun des folds, on a la valeur du critère (ROC_AUC) pour chacune des 15 valeurs différentes de la constante C. La variabilité sur chacune des 3 répétitions est due à l'aléa. Lorsque $C$ est très petite, on pénalise beaucoup et le critère ROC-AUC n'est pas bon. Lorsque $C$ augmente, à partir d'un moment on n'améliore pas la valeur du critère. 

Ce qui nous intéresse, pour chaque valeur de C, c'est la valeur du critère moyennée sur les cv répétitions.

In [ ]:
# Visualisation du critère global de CV 

score_boot = crit.mean(axis=0)
print(score_boot)

# Declaring the figure (width, height)
plt.figure(figsize=[15, 10])

# X-axis
#variables = np.logspace(-3, 5, 15)
#x_pos = np.arange(len(variables))

# Create bars
plt.bar(x_pos, score_boot, color = 'b', width = 0.25)
# Create names on the x-axis and title
plt.xticks(x_pos,variables)
plt.title('ROC-AUC-CV', fontsize=18)

# Show graphic
plt.show()


On voit bien que pour C=10^(-3), la pénalité est trop forte. On ne sélectionne aucune variable et on prédit au hasard (AUC-ROC=0,5).

Quand C devient grand, la pénalité diminue, et on sur-ajuste le modèle. 

La valeur optimale (qui donne le plus grand ROC_AUC moyen) est obtenue directement:

In [ ]:
C_opt = logreg_cv.C_[0]
print("C optimale =", C_opt)

### Exercice 10

Il faut à présent apprendre le modèle (de régression logistique pénalisée en nomre 1) avec ce C optimal (car pour l'instant on a 3 classifieurs différents construits avec cette valeur sur chacun des folds et de toutes façons on y a pas accès !). Ensuite, tracer les courbes ROC et PR et les comparer au classifieur sans pénalité.

In [ ]:
# Your answer

### Réponse

In [ ]:
# Régression logistique avec pénalisation norme 1 et constante optimale (choisie par CV)
logreg_l1Coptim = LogisticRegression(penalty='l1', solver='liblinear', C=C_opt, max_iter=1000)
# On estime les paramètres de ce modèle sur les données 
result_l1Coptim = logreg_l1Coptim.fit(X_train, Y_train)

Comparons le modèle complet sans pénalité avec le modèle pénalisé avec sélection automatique de la constante de pénalité

In [ ]:
# Comparons les classifieurs via leurs courbes ROC et les AUC-ROC

fig, ax = plt.subplots()
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')

# Sur le jeu de TEST, on reprend les scores prédits (probas que Y=1) 
scores_test_l1Coptim= logreg_l1Coptim.predict_proba(X_test)[:,1]
Y_pred_l1Coptim= logreg_l1Coptim.predict(X_test)
fpr, tpr, _ = roc_curve(Y_test, scores_test_l1Coptim) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, color='r',label=f"LogReg-L1-penalty-Copt - AUC = %0.2f" % auc(fpr, tpr))

# courbe ROC régression simple
fpr, tpr, _ = roc_curve(Y_test, scores_test) # on calcule les FPR et TPR pour tout un ensemble de seuils t non précisés
ax.plot(fpr, tpr, color='b',label=f"LogReg (no penalty) - AUC = %0.2f" % auc(fpr, tpr))

plt.legend(loc="lower right")
plt.show()

Les performances semblent identiques, mais la version pénalisée utilise beaucoup moins de variables. 

In [ ]:
# On compare maintenant les deux classifieurs via leurs courbes PR et les valeurs AUC-PR

fig, ax = plt.subplots()
ax.plot([0, 1], [1,0], 'k--')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

# Sur le jeu de TEST 
# pour la prediction pénalisée en norme 1
precision, recall, _ = precision_recall_curve(Y_test, scores_test_l1Coptim)
ax.plot(recall, precision, color='r',label=f"LogReg-L1-penalty-Coptim - AUC = %0.2f" % metrics.auc(recall,precision)) 

#  pour la prédiction sans pénalité 
precision, recall, _ = precision_recall_curve(Y_test, scores_test) 
ax.plot(recall, precision, color='b',label=f"LogReg (no penalty) - AUC = %0.2f" % metrics.auc(recall,precision))

plt.legend(loc="upper right")
plt.show()

## Aller plus loin

On peut passer un temps infini à trouver le meilleur modèle. Globalement, on peut travailler sur deux parties : 
1. les variables en entrée : garder tous les features ? aussi ceux qui sont très corrélés ? et pour les variables qualitatives, garder tous les dummies ? ou regrouper des catégories de faible taille ?
2. trouver les meilleurs hyperparamètres des modèles, en particulier par validation croisée excessive.

A vous de jouer, si vous avez encore un peu d'énergie :-)